In [1]:
import os
import shutil

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator


In [2]:
spark = SparkSession.builder \
    .appName("AirlinesDelay-SQL-Notebook") \
    .getOrCreate()

sc = spark.sparkContext


25/12/28 02:00:39 WARN Utils: Your hostname, spark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/12/28 02:00:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/28 02:00:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/28 02:00:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/28 02:00:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
INPUT_CSV = "data/airlines_delay.csv"

df_raw = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .csv(INPUT_CSV))

df_raw.printSchema()
df_raw.show(5, truncate=False)

root
 |-- Flight: double (nullable = true)
 |-- Time: double (nullable = true)
 |-- Length: double (nullable = true)
 |-- Airline: string (nullable = true)
 |-- AirportFrom: string (nullable = true)
 |-- AirportTo: string (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Class: integer (nullable = true)

+------+------+------+-------+-----------+---------+---------+-----+
|Flight|Time  |Length|Airline|AirportFrom|AirportTo|DayOfWeek|Class|
+------+------+------+-------+-----------+---------+---------+-----+
|2313.0|1296.0|141.0 |DL     |ATL        |HOU      |1        |0    |
|6948.0|360.0 |146.0 |OO     |COS        |ORD      |4        |0    |
|1247.0|1170.0|143.0 |B6     |BOS        |CLT      |3        |0    |
|31.0  |1410.0|344.0 |US     |OGG        |PHX      |6        |0    |
|563.0 |692.0 |98.0  |FL     |BMI        |ATL      |4        |0    |
+------+------+------+-------+-----------+---------+---------+-----+
only showing top 5 rows



In [4]:
df = df_raw

df = (df
      .withColumn("Time", F.col("Time").cast("int"))
      .withColumn("Length", F.col("Length").cast("int"))
      .withColumn("DayOfWeek", F.col("DayOfWeek").cast("int"))
      .withColumn("Class", F.col("Class").cast("int"))
      .withColumn("Airline", F.trim(F.col("Airline")))
      .withColumn("AirportFrom", F.trim(F.col("AirportFrom")))
      .withColumn("AirportTo", F.trim(F.col("AirportTo")))
)

df = df.dropna(subset=["Time","Length","DayOfWeek","Airline","AirportFrom","AirportTo","Class"]).cache()

print("Rows:", df.count())
df.groupBy("Class").count().orderBy("Class").show()

Rows: 539382
+-----+------+
|Class| count|
+-----+------+
|    0|299118|
|    1|240264|
+-----+------+



In [5]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
print("train:", train_df.count())
print("test :", test_df.count())

train_df.createOrReplaceTempView("train_flights")
test_df.createOrReplaceTempView("test_flights")
df.createOrReplaceTempView("flights")

train: 431452


test : 107930


### Advanced SQL Query 1 (Window): Airline × Hour + Moving Average

In [6]:
sql_airline_hour_ma = spark.sql("""
WITH hourly AS (
  SELECT
    Airline,
    CAST(FLOOR(Time/60) AS INT) AS hour,
    COUNT(*) AS flight_cnt,
    AVG(CAST(Class AS DOUBLE)) AS delay_rate
  FROM flights
  GROUP BY Airline, CAST(FLOOR(Time/60) AS INT)
),
with_ma AS (
  SELECT
    Airline,
    hour,
    flight_cnt,
    delay_rate,
    AVG(delay_rate) OVER (
      PARTITION BY Airline
      ORDER BY hour
      ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
    ) AS delay_rate_ma_3h
  FROM hourly
)
SELECT *
FROM with_ma
WHERE flight_cnt >= 30
ORDER BY Airline, hour
""")

sql_airline_hour_ma.show(50, truncate=False)


+-------+----+----------+-------------------+-------------------+
|Airline|hour|flight_cnt|delay_rate         |delay_rate_ma_3h   |
+-------+----+----------+-------------------+-------------------+
|9E     |5   |534       |0.25280898876404495|0.2588013861931402 |
|9E     |6   |1673      |0.2647937836222355 |0.29374637867421466|
|9E     |7   |1111      |0.36363636363636365|0.3150211379305551 |
|9E     |8   |998       |0.3166332665330661 |0.34814957821568054|
|9E     |9   |1340      |0.3641791044776119 |0.3576392113895242 |
|9E     |10  |1520      |0.39210526315789473|0.3784869156039619 |
|9E     |11  |1287      |0.3791763791763792 |0.4098543623056961 |
|9E     |12  |1606      |0.45828144458281445|0.4295062469797358 |
|9E     |13  |1461      |0.4510609171800137 |0.4361307476659942 |
|9E     |14  |1263      |0.3990498812351544 |0.44700662977475303|
|9E     |15  |1485      |0.4909090909090909 |0.4397412939972265 |
|9E     |16  |1442      |0.4292649098474341 |0.46427675025217496|
|9E     |1

### Advanced SQL Query 2 (CTE + Rank + Percentile): Origin bazında top rotalar

In [7]:
sql_top_routes_by_origin = spark.sql("""
WITH route_stats AS (
  SELECT
    AirportFrom,
    AirportTo,
    COUNT(*) AS flight_cnt,
    AVG(CAST(Class AS DOUBLE)) AS delay_rate,
    AVG(Length) AS avg_length,
    AVG(Time) AS avg_time
  FROM flights
  GROUP BY AirportFrom, AirportTo
),
ranked AS (
  SELECT
    *,
    DENSE_RANK() OVER (
      PARTITION BY AirportFrom
      ORDER BY delay_rate DESC
    ) AS rnk,
    percentile_approx(delay_rate, 0.90) OVER (PARTITION BY AirportFrom) AS p90_delay_rate_origin
  FROM route_stats
)
SELECT
  AirportFrom, AirportTo, flight_cnt, delay_rate, avg_length, avg_time, p90_delay_rate_origin
FROM ranked
WHERE flight_cnt >= 50 AND rnk <= 3
ORDER BY AirportFrom, delay_rate DESC
""")

sql_top_routes_by_origin.show(80, truncate=False)


+-----------+---------+----------+-------------------+------------------+-----------------+---------------------+
|AirportFrom|AirportTo|flight_cnt|delay_rate         |avg_length        |avg_time         |p90_delay_rate_origin|
+-----------+---------+----------+-------------------+------------------+-----------------+---------------------+
|ABE        |DTW      |85        |0.47058823529411764|112.58823529411765|687.435294117647 |0.47058823529411764  |
|ABE        |ORD      |147       |0.32653061224489793|140.06802721088437|635.2312925170068|0.47058823529411764  |
|ABI        |DFW      |211       |0.15165876777251186|50.639810426540286|623.6729857819905|0.15165876777251186  |
|ABQ        |SAN      |59        |0.8305084745762712 |117.37288135593221|827.0338983050848|0.78125              |
|ABY        |ATL      |87        |0.25287356321839083|56.632183908045974|665.9425287356322|0.25287356321839083  |
|ACT        |DFW      |60        |0.2833333333333333 |45.0              |641.5          

### “SQL used in application”: Train’den feature üret → Train/Test’e join

In [8]:
airline_feats = spark.sql("""
SELECT
  Airline,
  COUNT(*) AS airline_flight_cnt_train,
  AVG(CAST(Class AS DOUBLE)) AS airline_delay_rate_train
FROM train_flights
GROUP BY Airline
""")

route_feats = spark.sql("""
SELECT
  AirportFrom,
  AirportTo,
  COUNT(*) AS route_flight_cnt_train,
  AVG(CAST(Class AS DOUBLE)) AS route_delay_rate_train
FROM train_flights
GROUP BY AirportFrom, AirportTo
""")

airline_feats.show(5, truncate=False)
route_feats.show(5, truncate=False)


+-------+------------------------+------------------------+
|Airline|airline_flight_cnt_train|airline_delay_rate_train|
+-------+------------------------+------------------------+
|UA     |22022                   |0.323540096267369       |
|AA     |36509                   |0.39023254539976443     |
|EV     |22357                   |0.4018875519971374      |
|B6     |14524                   |0.4666069953180942      |
|DL     |48618                   |0.45069727261508086     |
+-------+------------------------+------------------------+
only showing top 5 rows



+-----------+---------+----------------------+----------------------+
|AirportFrom|AirportTo|route_flight_cnt_train|route_delay_rate_train|
+-----------+---------+----------------------+----------------------+
|SNA        |PHX      |350                   |0.5142857142857142    |
|ORD        |PDX      |127                   |0.4251968503937008    |
|MCI        |MKE      |73                    |0.6164383561643836    |
|SMF        |BUR      |175                   |0.6                   |
|PHL        |MCO      |375                   |0.448                 |
+-----------+---------+----------------------+----------------------+
only showing top 5 rows



In [9]:
global_delay = train_df.select(F.avg(F.col("Class").cast("double")).alias("g")).collect()[0]["g"]
print("Global train delay rate:", global_delay)

train_fe = (train_df
    .join(airline_feats, on="Airline", how="left")
    .join(route_feats, on=["AirportFrom","AirportTo"], how="left")
    .fillna({
        "airline_flight_cnt_train": 0,
        "airline_delay_rate_train": global_delay,
        "route_flight_cnt_train": 0,
        "route_delay_rate_train": global_delay
    })
).cache()

test_fe = (test_df
    .join(airline_feats, on="Airline", how="left")
    .join(route_feats, on=["AirportFrom","AirportTo"], how="left")
    .fillna({
        "airline_flight_cnt_train": 0,
        "airline_delay_rate_train": global_delay,
        "route_flight_cnt_train": 0,
        "route_delay_rate_train": global_delay
    })
).cache()

train_fe.select("Airline","airline_delay_rate_train","route_delay_rate_train","Class").show(5, truncate=False)


Global train delay rate: 0.4456393758749525


+-------+------------------------+----------------------+-----+
|Airline|airline_delay_rate_train|route_delay_rate_train|Class|
+-------+------------------------+----------------------+-----+
|US     |0.334917548786635       |0.5142857142857142    |0    |
|US     |0.334917548786635       |0.5142857142857142    |0    |
|US     |0.334917548786635       |0.5142857142857142    |0    |
|US     |0.334917548786635       |0.5142857142857142    |0    |
|US     |0.334917548786635       |0.5142857142857142    |0    |
+-------+------------------------+----------------------+-----+
only showing top 5 rows



### Mini “Application”: Bu SQL feature’larla model eğitip metrik bas (raporda çok iyi durur)

In [10]:
label_col = "Class"
cat_cols = ["Airline", "AirportFrom", "AirportTo"]
num_cols = [
    "Time", "Length", "DayOfWeek",
    "airline_flight_cnt_train", "airline_delay_rate_train",
    "route_flight_cnt_train", "route_delay_rate_train"
]

indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in cat_cols]
encoder = OneHotEncoder(
    inputCols=[f"{c}_idx" for c in cat_cols],
    outputCols=[f"{c}_ohe" for c in cat_cols],
    handleInvalid="keep"
)
assembler = VectorAssembler(inputCols=num_cols + [f"{c}_ohe" for c in cat_cols], outputCol="features")

lr = LogisticRegression(featuresCol="features", labelCol=label_col, maxIter=50)

pipe = Pipeline(stages=indexers + [encoder, assembler, lr])
m = pipe.fit(train_fe)
pred = m.transform(test_fe)

acc_eval = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol="prediction", metricName="accuracy")
f1_eval  = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol="prediction", metricName="f1")
auc_eval = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

print("SQL-feature LR -> Accuracy:", acc_eval.evaluate(pred))
print("SQL-feature LR -> F1      :", f1_eval.evaluate(pred))
print("SQL-feature LR -> AUC     :", auc_eval.evaluate(pred))

pred.groupBy("Class").pivot("prediction", [0.0, 1.0]).count().na.fill(0).show()


SQL-feature LR -> Accuracy: 0.6517650328916891


SQL-feature LR -> F1      : 0.6450325780756409


SQL-feature LR -> AUC     : 0.6962157855666282


+-----+-----+-----+
|Class|  0.0|  1.0|
+-----+-----+-----+
|    1|23738|24254|
|    0|46091|13847|
+-----+-----+-----+



In [11]:
OUT_SQL1 = "outputs/sql/airline_hour_ma"
OUT_SQL2 = "outputs/sql/top_routes_by_origin"
OUT_FEATS_AIRLINE = "outputs/sql_features/airline_feats"
OUT_FEATS_ROUTE   = "outputs/sql_features/route_feats"

for p in [OUT_SQL1, OUT_SQL2, OUT_FEATS_AIRLINE, OUT_FEATS_ROUTE]:
    if os.path.exists(p):
        shutil.rmtree(p)


In [12]:
sql_airline_hour_ma.coalesce(1).write.mode("overwrite").option("header", True).csv(OUT_SQL1)
sql_top_routes_by_origin.coalesce(1).write.mode("overwrite").option("header", True).csv(OUT_SQL2)

airline_feats.coalesce(1).write.mode("overwrite").option("header", True).csv(OUT_FEATS_AIRLINE)
route_feats.coalesce(1).write.mode("overwrite").option("header", True).csv(OUT_FEATS_ROUTE)

print("Saved:")
print(" -", OUT_SQL1)
print(" -", OUT_SQL2)
print(" -", OUT_FEATS_AIRLINE)
print(" -", OUT_FEATS_ROUTE)


Saved:
 - outputs/sql/airline_hour_ma
 - outputs/sql/top_routes_by_origin
 - outputs/sql_features/airline_feats
 - outputs/sql_features/route_feats
